# Goal

The objective of this notebook is to demonstrate and visualize the computation of the pareto frontier.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd


from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.SyntheticFunctions.Hypersphere import Hypersphere

from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Optimizers.ParetoFrontier import ParetoFrontier

from mlos.Spaces import ContinuousDimension, DiscreteDimension, Point, SimpleHypergrid

hypersphere_radius = 10

objective_function_config = Point(
    implementation=Hypersphere.__name__,
    hypersphere_config=Point(
        num_objectives=2,
        minimize='none',
        radius=hypersphere_radius
    )
)

objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config=objective_function_config)
optimization_problem = objective_function.default_optimization_problem
optimizer = BayesianOptimizerFactory().create_local_optimizer(
    optimization_problem=optimization_problem
)

01/22/2021 00:22:11 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizerFactory.py:  37 -    create_local_optimizer() ] Optimizer config not specified. Using default.


In [3]:
optimizer.optimizer_config

{
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 10,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.max_depth": 0,
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.min_samples_spli

In [ ]:
num_iterations = 100
for i in range(num_iterations):
    print(i)
    config = optimizer.suggest()
    value = objective_function.evaluate_point(config)
    optimizer.register(config.to_dataframe(), value.to_dataframe())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [62]:
# Now let's plot the existing pareto frontier, along with some predictions.
#
params_df, objectives_df, context_df = optimizer.get_all_observations()

# Let's create pareto frontiers over time so that we can plot them.
#
pareto_frontiers_over_time = []
pareto_dfs_over_time = []
for i in objectives_df.index:
    pareto_frontier = ParetoFrontier(optimization_problem=optimization_problem, objectives_df=objectives_df[:i])
    pareto_frontiers_over_time.append(pareto_frontier)
    pareto_dfs_over_time.append(pareto_frontier.pareto_df)
objectives_over_time = [objectives_df[:i] for i in objectives_df.index]

In [63]:
# Let's add a leading and trailing row to the frontier to make the volume clearer.
#
augmented_pareto_dfs_over_time = []
for pareto_df in pareto_dfs_over_time:
    max_y0 = pareto_df['y0'].max()
    max_y1 = pareto_df['y1'].max()
    last_row = pd.DataFrame({'y0': [0], 'y1': [pareto_df['y1'].max()]})
    first_row = pd.DataFrame({'y0': [pareto_df['y0'].max()], 'y1': [0]})
    augmented_pareto_df = pd.concat([first_row, pareto_df, last_row], ignore_index=True)
    augmented_pareto_dfs_over_time.append(augmented_pareto_df)

pareto_dfs_over_time = augmented_pareto_dfs_over_time

In [64]:
augmented_pareto_dfs_over_time[10]

,y0,y1
0,6.842249,0.000000
1,6.842249,4.176298
2,4.672388,4.688695
3,3.253044,7.370692
4,0.000000,7.370692


In [65]:
len(pareto_dfs_over_time)

178

In [66]:
len(augmented_pareto_dfs_over_time)

178

In [68]:
import plotly.graph_objs as go

fig = go.Figure()

for num_samples in (100, 10000, 1000000):
    for alpha in (0.05, 0.01):
        lower_bounds = []
        upper_bounds = []
        iterations = []
        for i, pareto_frontier in enumerate(pareto_frontiers_over_time[1:]):
            volume_estimator = pareto_frontier.approximate_pareto_volume(num_samples=num_samples)
            lower_bound, upper_bound = volume_estimator.get_two_sided_confidence_interval_on_pareto_volume(alpha=alpha)
            iterations.append(i)
            lower_bounds.append(lower_bound)
            upper_bounds.append(upper_bound)

        fig.add_trace(go.Scatter(
            x=iterations + iterations[::-1],
            y=upper_bounds + lower_bounds[::-1],
            fill='toself',
            name=f'{(1-alpha)*100:.0f}CI from {num_samples} samples'
        ))

fig.update_layout(
    title='Pareto Frontier Volume vs. Iteration Number',
    xaxis_title="Iteration Number",
    yaxis_title='Pareto Frontier Volume',
    showlegend=True
)
fig.show()    

In [30]:
import numpy as np
if True:
    # Let's create the true pareto frontier so that we can plot it.
    #
    objective_function.parameter_space.dimensions
    theta0 = objective_function.parameter_space['theta0'].linspace(20)
    theta1 = objective_function.parameter_space['theta1'].linspace(20)
    theta0, theta1 = np.meshgrid(theta0, theta1)
    optimum_params_df = pd.DataFrame({
        'theta0': theta0.flatten(),
        'theta1': theta1.flatten(),
    })
    optimum_params_df['radius'] = hypersphere_radius
    optimum_objectives_df = objective_function.evaluate_dataframe(optimum_params_df)
    true_pareto_frontier = ParetoFrontier(optimization_problem, optimum_objectives_df)
    true_pareto_df = true_pareto_frontier.pareto_df

In [14]:
pareto_dfs_over_time

,y0,y1
14,7.963996,4.169620
24,7.231773,6.870784
25,6.835072,6.998457
36,6.469320,7.304424
41,5.989545,8.007723
29,4.521421,8.644194


In [70]:
import plotly.graph_objects as go



# Create figure
fig = go.Figure(
    data=[
        go.Scatter(
            x=objectives_over_time[0]['y0'],
            y=objectives_over_time[0]['y1'],
            mode="markers",
            marker=dict(color="blue", size=10),
            name='observed objectives'
        ),
        go.Scatter(
            x=true_pareto_df['y0'],
            y=true_pareto_df['y1'],
            name='true pareto'
            #mode="markers",
            #marker=dict(color="red", size=10)
        )
    ],
    layout=go.Layout(
        xaxis=dict(range=[0, 10], autorange=False, zeroline=False),
        yaxis=dict(range=[0, 10], autorange=False, zeroline=False),
        title_text="Pareto over time",
        hovermode="closest",
        updatemenus=[dict(type="buttons", buttons=[dict(label="Play", method="animate", args=[None])])],
        width=700,
        height=600
    ),
    frames=[
        go.Frame(
            data=[
                #go.Scatter(
                #    x=true_pareto_df['y0'],
                #    y=true_pareto_df['y1'],
                #    name='true pareto',
                #    mode="lines",
                #    #marker=dict(color="red", size=10)
                #),
                go.Scatter(
                    x=pareto_dfs_over_time[i]['y0'],
                    y=pareto_dfs_over_time[i]['y1'],
                    name='estimated pareto',
                    mode='lines+markers'
                ),
                go.Scatter(
                    x=objectives_over_time[i]['y0'],
                    y=objectives_over_time[i]['y1'],
                    mode="markers",
                    marker=dict(color="blue", size=10),
                    name='observed objectives'
                ),
                
            ]
        )
        for i in objectives_df.index
    ]
)

fig.show()

In [ ]:
# We can also compute some speculative pareto I think :)
#
objective_function.parameter_space.dimensions
theta0 = objective_function.parameter_space['theta0'].linspace(40)
theta1 = objective_function.parameter_space['theta1'].linspace(40)
radius = objective_function.parameter_space['radius'].linspace(40)
theta0, theta1, radius = np.meshgrid(theta0, theta1, radius)

params_df = pd.DataFrame({
    'theta0': theta0.flatten(),
    'theta1': theta1.flatten(),
    'radius': radius.flatten()
})


features_df = optimization_problem.construct_feature_dataframe(params_df)
mo_predictions = optimizer.surrogate_model.predict(features_df)

predicted_objectives_df = pd.DataFrame()
for objective_name, prediction in mo_predictions:
    prediction_df = prediction.get_dataframe()
    predicted_objectives_df[objective_name] = prediction_df['predicted_value']

speculative_pareto_df = ParetoFrontier(optimization_problem, predicted_objectives_df).pareto_df
empirical_pareto_df = ParetoFrontier(optimization_problem, objectives_df).pareto_df

In [25]:
empirical_pareto_df

,y0,y1
14,7.963996,4.169620
24,7.231773,6.870784
25,6.835072,6.998457
36,6.469320,7.304424
41,5.989545,8.007723
29,4.521421,8.644194
45,4.167211,8.840191


In [22]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure(
    data=[
        go.Scatter(
            x=speculative_pareto_df['y0'],
            y=speculative_pareto_df['y1'],
            name='speculative_pareto',
            mode='markers'
        ),
        go.Scatter(
            x=empirical_pareto_df['y0'],
            y=empirical_pareto_df['y1'],
            name='empirical pareto',
            mode='markers'
        ),
        go.Scatter(
            x=true_pareto_df['y0'],
            y=true_pareto_df['y1'],
            name='true pareto'
            #mode="markers",
            #marker=dict(color="red", size=10)
        )
    ],
    layout=go.Layout(
        xaxis=dict(range=[0, 10], autorange=False, zeroline=False),
        yaxis=dict(range=[0, 10], autorange=False, zeroline=False),
        title_text="Pareto over time",
        hovermode="closest",
        width=800,
        height=800
    )
)

fig.show()